<a href="https://colab.research.google.com/github/IsabellaAGomes/redundancy_identification/blob/main/step4_train_and_testing_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat May 25 12:04:42 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              52W / 400W |   6679MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
!python --version ou !python -V

Python 3.10.12


In [ ]:
#Instalação da biblioteca de Nuralmind
!pip install transformers

In [ ]:
#Importando bibliotecas padrões + módulos de classificação da bibloteca do Bert, módulo de tokenização e módulo de pipeline
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertForSequenceClassification, AutoTokenizer, BertModel
from tqdm import tqdm

print(pd.__version__)
print(np.__version__)
print(torch.__version__)

2.0.3
1.25.2
2.3.0+cu121


In [ ]:
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   6134 MiB |  27625 MiB | 292607 GiB | 292601 GiB |\n|       from large pool |   6134 MiB |  27609 MiB | 292456 GiB | 292450 GiB |\n|       from small pool |      0 MiB |     17 MiB |    150 GiB |    150 GiB |\n|---------------------------------------------------------------------------|\n| Active memory         |   6134 MiB |  27625 MiB | 292607 GiB | 292601 GiB |\n|       from large pool |   6134 MiB |  27609 MiB |

In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb=8'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [ ]:
#Flags de controle e hiperparâmetros
MAX_LENGTH = 512
BATCH_SIZE = 16
#NOTA: USAR BATCH_SIZE = 8 PARA EXPERIMENTOS QUE TREINAM COM O CONJUNTO INTEIRO
POS_PARAMETER = True
GRAD_PARAMETER = True
Execucao = "TestModel"
model_name = 'experiment_1_and_2_epoch_2'
conjunto_treino = "assin-trv1-pos.xml"
conjunto_teste2 = "assin2-t2-pos.xml"
conjunto_teste = "assin-t1-pos.xml"
#Como sugerido pelo artigo original do Bert, o modelo foi treinado usando 3 épocas.
epochs = 3

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
print("Conferindo a unidade de processamento: ", device)

Conferindo a unidade de processamento:  cuda:0


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

In [ ]:
#Dataloader é um construtor de dataset do PyTorch, que permite que faça pré processamento de dados enquanto os dados estão sendo carregados.
#Vamos definir uma classe que será consumida pelo Dataloader quando alimentarmos o modelo
#com os dados deste treino.
# Na classe abaixo, os atributos e labels são assigned e passados para a cpu/gpu
class TextDataset(Dataset):

  def __init__(self, X, y):

    self.X = X
    self.X = self.X.to(device)

    self.y = y
    self.y = self.y.to(device)

    self.len = len(y)

#É necessário redefinir o len
  def __len__(self):
    return self.len

#Define como as instâncias serão passadas para o modelo de treino
  def __getitem__(self, idx):
    return self.X[:,idx], self.y[idx]


In [ ]:
!pip install datasets
from datasets import load_metric
metric1 = load_metric("precision")
metric2 = load_metric("recall")
metric3 = load_metric("f1")


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for precision contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/precision/precision.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for recall contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/recall/recall.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric fro

In [ ]:
def TrainTestModel(train_loader, val_loader, test_loader):

  #Setup do treino
  steps_per_epoch = len(train_loader)
  #epoch_validation_samples = 50

  epoch_validation_samples = len(val_loader)
  epoch_test_samples = len(test_loader)

  print("Tamanho Treino: " + str(len(train_loader)))
  print("Steps per epoch: " + str(steps_per_epoch))
  print("Tamanho Validação: " + str(epoch_validation_samples))
  print("Tamanho Test: " + str(epoch_test_samples))

  model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased').to(device)
  for param in model.base_model.parameters():
    param.requires_grad = GRAD_PARAMETER

  loss_func = torch.nn.CrossEntropyLoss()
  optim = torch.optim.Adam(model.parameters())

  acc_calc = lambda output, labels: (labels == output.argmax(axis=1)).sum()
  scheduler = torch.optim.lr_scheduler.ExponentialLR(optim, 0.9997)

  epoch_metadata = []

  for i in range(epochs):
    val_iteracoes = 0
    test_iteracoes = 0
    train_iteracoes = 0
    num_train_examples = 0
    num_val_examples = 0
    num_test_examples = 0

    train_hits_softmax = 0
    train_hits_sigmoid = 0
    val_hits_softmax = 0
    val_hits_sigmoid = 0
    test_hits_softmax = 0
    test_hits_sigmoid = 0

    train_precision_softmax = 0
    train_recall_softmax = 0
    train_f1_softmax = 0
    train_f1_macro_softmax = 0
    train_precision_sigmoid = 0
    train_recall_sigmoid = 0
    train_f1_sigmoid = 0
    train_f1_macro_sigmoid = 0

    val_precision_softmax = 0
    val_recall_softmax = 0
    val_f1_softmax = 0
    val_f1_macro_softmax = 0
    val_precision_sigmoid = 0
    val_recall_sigmoid = 0
    val_f1_sigmoid = 0
    val_f1_macro_sigmoid = 0

    test_precision_softmax = 0
    test_recall_softmax = 0
    test_f1_softmax = 0
    test_f1_macro_softmax = 0
    test_precision_sigmoid = 0
    test_recall_sigmoid = 0
    test_f1_sigmoid = 0
    test_f1_macro_sigmoid = 0


    #Barras para monitorar o treino
    train_bar = tqdm(total=steps_per_epoch, desc=f'Train', unit='steps', position=0, leave=True)
    val_bar = tqdm(total=epoch_validation_samples, desc=f'Val', unit='samples', position=0, leave=True)
    test_bar = tqdm(total=epoch_test_samples, desc=f'Test', unit='samples', position=0, leave=True)

    for batch_number, (features, labels) in enumerate(train_loader):

      train_running_loss = 0

      model.train()

      input_ids, input_masks, input_token_type_ids = features[:, 0, :], features[:, 1, :], features[:, 2, :]
      input_masks = input_masks[..., :1]
      input_masks = torch.squeeze(input_masks, 2)
      input_token_type_ids = input_token_type_ids[..., :1]
      input_token_type_ids = torch.squeeze(input_token_type_ids, 2)

      saida_modelo = model(inputs_embeds=input_ids, attention_mask=input_masks, token_type_ids=input_token_type_ids.long(), labels=labels.long())
      #saida_modelo = model(inputs_embeds=input_ids, attention_mask=input_masks, labels=labels.long())

      loss, logits = saida_modelo[0], saida_modelo[1]

      optim.zero_grad()
      loss.backward(retain_graph=True)

#      loss.backward()
      optim.step()

      train_running_loss += loss.item()

      softmax_predictions = torch.nn.functional.softmax(logits, dim=1)
      sigmoid_predictions = torch.nn.functional.sigmoid(logits)
      train_hits_softmax += acc_calc(softmax_predictions, labels)
      train_hits_sigmoid += acc_calc(sigmoid_predictions, labels)

      train_precision_softmax += metric1.compute(predictions=softmax_predictions.argmax(axis=1), references=labels)["precision"]
      train_recall_softmax += metric2.compute(predictions=softmax_predictions.argmax(axis=1), references=labels)["recall"]
      train_f1_softmax += metric3.compute(predictions=softmax_predictions.argmax(axis=1), references=labels)["f1"]
      train_f1_macro_softmax += metric3.compute(predictions=softmax_predictions.argmax(axis=1), references=labels,average ="macro")["f1"]

      train_precision_sigmoid += metric1.compute(predictions=sigmoid_predictions.argmax(axis=1), references=labels)["precision"]
      train_recall_sigmoid += metric2.compute(predictions=sigmoid_predictions.argmax(axis=1), references=labels)["recall"]
      train_f1_sigmoid += metric3.compute(predictions=sigmoid_predictions.argmax(axis=1), references=labels)["f1"]
      train_f1_macro_sigmoid += metric3.compute(predictions=sigmoid_predictions.argmax(axis=1), references=labels,average ="macro")["f1"]

      train_bar.update(1)

      num_train_examples += features.shape[0]
      train_iteracoes = train_iteracoes + 1

      scheduler.step()

      if (batch_number + 1) % steps_per_epoch == 0:
        train_bar.close()
        break

    model.eval()

    for batch_number, (features, labels) in enumerate(val_loader):
      with torch.no_grad():

        val_running_loss = 0

        input_ids, input_masks, input_token_type_ids = features[:, 0, :], features[:, 1, :], features[:, 2, :]
        input_masks = input_masks[..., :1]
        input_masks = torch.squeeze(input_masks, 2)
        input_token_type_ids = input_token_type_ids[..., :1]
        input_token_type_ids = torch.squeeze(input_token_type_ids, 2)

        #saida_modelo = model(input_ids, input_masks, labels=labels.long())
        saida_modelo = model(inputs_embeds=input_ids, attention_mask=input_masks, token_type_ids=input_token_type_ids.long(), labels=labels.long())

        loss, logits = saida_modelo[0], saida_modelo[1]
        val_running_loss += loss.item()

        softmax_predictions = torch.nn.functional.softmax(logits, dim=1)
        sigmoid_predictions = torch.nn.functional.sigmoid(logits)
        val_hits_softmax += acc_calc(softmax_predictions, labels)
        val_hits_sigmoid += acc_calc(sigmoid_predictions, labels)

        val_precision_softmax += metric1.compute(predictions=softmax_predictions.argmax(axis=1), references=labels)["precision"]
        val_recall_softmax += metric2.compute(predictions=softmax_predictions.argmax(axis=1), references=labels)["recall"]
        val_f1_softmax += metric3.compute(predictions=softmax_predictions.argmax(axis=1), references=labels)["f1"]
        val_f1_macro_softmax += metric3.compute(predictions=softmax_predictions.argmax(axis=1), references=labels,average ="macro")["f1"]

        val_precision_sigmoid += metric1.compute(predictions=sigmoid_predictions.argmax(axis=1), references=labels)["precision"]
        val_recall_sigmoid += metric2.compute(predictions=sigmoid_predictions.argmax(axis=1), references=labels)["recall"]
        val_f1_sigmoid += metric3.compute(predictions=sigmoid_predictions.argmax(axis=1), references=labels)["f1"]
        val_f1_macro_sigmoid += metric3.compute(predictions=sigmoid_predictions.argmax(axis=1), references=labels,average ="macro")["f1"]


        num_val_examples += features.shape[0]
        val_iteracoes = val_iteracoes + 1

        val_bar.update(1)

        if (batch_number + 1) % epoch_validation_samples == 0:
          val_bar.close()
          break

    model.save_pretrained(f'experiment_1_and_2_epoch_{i}')

    model.eval()

    for batch_number, (features, labels) in enumerate(test_loader):
      with torch.no_grad():

        test_running_loss = 0

        input_ids, input_masks, input_token_type_ids = features[:, 0, :], features[:, 1, :], features[:, 2, :]
        input_masks = input_masks[..., :1]
        input_masks = torch.squeeze(input_masks, 2)
        input_token_type_ids = input_token_type_ids[..., :1]
        input_token_type_ids = torch.squeeze(input_token_type_ids, 2)

        #saida_modelo = model(input_ids, input_masks, labels=labels.long())
        saida_modelo = model(inputs_embeds=input_ids, attention_mask=input_masks, token_type_ids=input_token_type_ids.long(), labels=labels.long())

        loss, logits = saida_modelo[0], saida_modelo[1]
        test_running_loss += loss.item()

        softmax_predictions = torch.nn.functional.softmax(logits, dim=1)
        sigmoid_predictions = torch.nn.functional.sigmoid(logits)
        test_hits_softmax += acc_calc(softmax_predictions, labels)
        test_hits_sigmoid += acc_calc(sigmoid_predictions, labels)

        test_precision_softmax += metric1.compute(predictions=softmax_predictions.argmax(axis=1), references=labels)["precision"]
        test_recall_softmax += metric2.compute(predictions=softmax_predictions.argmax(axis=1), references=labels)["recall"]
        test_f1_softmax += metric3.compute(predictions=softmax_predictions.argmax(axis=1), references=labels)["f1"]
        test_f1_macro_softmax += metric3.compute(predictions=softmax_predictions.argmax(axis=1), references=labels,average ="macro")["f1"]

        test_precision_sigmoid += metric1.compute(predictions=sigmoid_predictions.argmax(axis=1), references=labels)["precision"]
        test_recall_sigmoid += metric2.compute(predictions=sigmoid_predictions.argmax(axis=1), references=labels)["recall"]
        test_f1_sigmoid += metric3.compute(predictions=sigmoid_predictions.argmax(axis=1), references=labels)["f1"]
        test_f1_macro_sigmoid += metric3.compute(predictions=sigmoid_predictions.argmax(axis=1), references=labels,average ="macro")["f1"]

        num_test_examples += features.shape[0]
        test_iteracoes = test_iteracoes + 1

        test_bar.update(1)

        if (batch_number + 1) % epoch_test_samples == 0:
          test_bar.close()
          break

#    train_acc = torch.true_divide(train_hits, num_train_examples)
#    val_acc = torch.true_divide(val_hits, num_val_examples)
#    test_acc = torch.true_divide(test_hits, num_test_examples)

    train_acc_softmax = torch.true_divide(train_hits_softmax, num_train_examples)
    train_acc_sigmoid = torch.true_divide(train_hits_sigmoid, num_train_examples)
    val_acc_softmax = torch.true_divide(val_hits_softmax, num_val_examples)
    val_acc_sigmoid = torch.true_divide(val_hits_sigmoid, num_val_examples)
    test_acc_softmax = torch.true_divide(test_hits_softmax, num_test_examples)
    test_acc_sigmoid = torch.true_divide(test_hits_sigmoid, num_test_examples)


    train_precision_softmax_final = torch.true_divide(train_precision_softmax, train_iteracoes)
    train_recall_softmax_final = torch.true_divide(train_recall_softmax, train_iteracoes)
    train_f1_softmax_final = torch.true_divide(train_f1_softmax, train_iteracoes)
    train_f1_macro_softmax_final = torch.true_divide(train_f1_macro_softmax, train_iteracoes)

    train_precision_sigmoid_final = torch.true_divide(train_precision_sigmoid, train_iteracoes)
    train_recall_sigmoid_final = torch.true_divide(train_recall_sigmoid, train_iteracoes)
    train_f1_sigmoid_final = torch.true_divide(train_f1_sigmoid, train_iteracoes)
    train_f1_macro_sigmoid_final = torch.true_divide(train_f1_macro_sigmoid, train_iteracoes)

    val_precision_softmax_final = torch.true_divide(val_precision_softmax, val_iteracoes)
    val_recall_softmax_final = torch.true_divide(val_recall_softmax, val_iteracoes)
    val_f1_softmax_final = torch.true_divide(val_f1_softmax, val_iteracoes)
    val_f1_macro_softmax_final = torch.true_divide(val_f1_macro_softmax, val_iteracoes)

    val_precision_sigmoid_final = torch.true_divide(val_precision_sigmoid, val_iteracoes)
    val_recall_sigmoid_final = torch.true_divide(val_recall_sigmoid, val_iteracoes)
    val_f1_sigmoid_final = torch.true_divide(val_f1_sigmoid, val_iteracoes)
    val_f1_macro_sigmoid_final = torch.true_divide(val_f1_macro_sigmoid, val_iteracoes)

    test_precision_softmax_final = torch.true_divide(test_precision_softmax, test_iteracoes)
    test_recall_softmax_final = torch.true_divide(test_recall_softmax, test_iteracoes)
    test_f1_softmax_final = torch.true_divide(test_f1_softmax, test_iteracoes)
    test_f1_macro_softmax_final = torch.true_divide(test_f1_macro_softmax, test_iteracoes)

    test_precision_sigmoid_final = torch.true_divide(test_precision_sigmoid, test_iteracoes)
    test_recall_sigmoid_final = torch.true_divide(test_recall_sigmoid, test_iteracoes)
    test_f1_sigmoid_final = torch.true_divide(test_f1_sigmoid, test_iteracoes)
    test_f1_macro_sigmoid_final = torch.true_divide(test_f1_macro_sigmoid, test_iteracoes)

    print(f"EPOCH SUMMARY - {i + 1}  TRAIN: \t Train loss: {train_running_loss} ")
    print(f"EPOCH SUMMARY - {i + 1}  TRAIN SOFTMAX: \t Train Acc softmax: {train_acc_softmax} \t Train precision softmax: {train_precision_softmax_final} \t Train recall softmax: {train_recall_softmax_final}  \t Train F1 softmax: {train_f1_softmax_final} \t Train F1 macro softmax: {train_f1_macro_softmax_final} ")
    print(f"EPOCH SUMMARY - {i + 1}  TRAIN SIGMOID: \t Train Acc sigmoid: {train_acc_sigmoid} \t Train precision sigmoid: {train_precision_sigmoid_final} \t Train recall sigmoid: {train_recall_sigmoid_final}  \t Train F1 sigmoid: {train_f1_sigmoid_final} \t Train F1 macro sigmoid: {train_f1_macro_sigmoid_final} ")

    print(f"EPOCH SUMMARY - {i + 1}  VALIDAÇÃO: \t Val loss: {val_running_loss} ")
    print(f"EPOCH SUMMARY - {i + 1}  VALIDAÇÃO SOFTMAX: \t Val Acc softmax: {val_acc_softmax} \t Val precision softmax: {val_precision_softmax_final} \t Val recall softmax: {val_recall_softmax_final}  \t Val F1 softmax: {val_f1_softmax_final} \t Val F1 macro softmax: {val_f1_macro_softmax_final} ")
    print(f"EPOCH SUMMARY - {i + 1}  VALIDAÇÃO SIGMOID: \t Val Acc sigmoid: {val_acc_sigmoid} \t Val precision sigmoid: {val_precision_sigmoid_final} \t Val recall sigmoid: {val_recall_sigmoid_final}  \t Val F1 sigmoid: {val_f1_sigmoid_final} \t Val F1 macro sigmoid: {val_f1_macro_sigmoid_final} ")

    print(f"EPOCH SUMMARY - {i + 1}  TEST: \t test loss: {test_running_loss} ")
    print(f"EPOCH SUMMARY - {i + 1}  TEST SOFTMAX: \t Test Acc softmax: {test_acc_softmax} \t Test precision softmax: {test_precision_softmax_final} \t Test recall softmax: {test_recall_softmax_final}  \t Test F1 softmax: {test_f1_softmax_final} \t Test F1 macro softmax: {test_f1_macro_softmax_final} ")
    print(f"EPOCH SUMMARY - {i + 1}  TEST SIGMOID: \t Test Acc sigmoid: {test_acc_sigmoid} \t Test precision sigmoid: {test_precision_sigmoid_final} \t Test recall sigmoid: {test_recall_sigmoid_final}  \t Test F1 sigmoid: {test_f1_sigmoid_final} \t Test F1 macro sigmoid: {test_f1_macro_sigmoid_final} ")



  #  print(f"t Test loss: {test_running_loss} \t Test Acc: {test_acc}")
  #  print(f"EPOCH SUMMARY - {i + 1} \t Train loss: {train_running_loss} \t Train Acc: {train_acc} \t Val loss: {val_running_loss} \t Val Acc: {val_acc} \t Test loss: {test_running_loss} \t Test Acc: {test_acc}")

#    print(f"EPOCH SUMMARY - {i + 1} \t Train loss: {train_running_loss} \t Train Acc: {train_acc} \t Test loss: {test_running_loss} \t Test Acc: {test_acc}")

  return train_acc_softmax

In [ ]:
def TrainModel(train_loader):

  #Setup do treino
  steps_per_epoch = len(train_loader)
  #epoch_validation_samples = 50

  print("Tamanho Treino: " + str(len(train_loader)))
  print("Steps per epoch: " + str(steps_per_epoch))
  print("Grad: " + str(GRAD_PARAMETER))

  model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased').to(device)
  for param in model.base_model.parameters():
    param.requires_grad = GRAD_PARAMETER

  loss_func = torch.nn.CrossEntropyLoss()
  optim = torch.optim.Adam(model.parameters())

  acc_calc = lambda output, labels: (labels == output.argmax(axis=1)).sum()
  scheduler = torch.optim.lr_scheduler.ExponentialLR(optim, 0.9997)

  epoch_metadata = []

  for i in range(epochs):
    train_iteracoes = 0
    num_train_examples = 0
    train_hits_softmax = 0
    train_hits_sigmoid = 0

    train_precision_softmax = 0
    train_recall_softmax = 0
    train_f1_softmax = 0
    train_f1_macro_softmax = 0
    train_precision_sigmoid = 0
    train_recall_sigmoid = 0
    train_f1_sigmoid = 0
    train_f1_macro_sigmoid = 0


    #Barras para monitorar o treino
    train_bar = tqdm(total=steps_per_epoch, desc=f'Train', unit='steps', position=0, leave=True)

    for batch_number, (features, labels) in enumerate(train_loader):

      train_running_loss = 0

      model.train()

      input_ids, input_masks, input_token_type_ids = features[:, 0, :], features[:, 1, :], features[:, 2, :]
      input_masks = input_masks[..., :1]
      input_masks = torch.squeeze(input_masks, 2)
      input_token_type_ids = input_token_type_ids[..., :1]
      input_token_type_ids = torch.squeeze(input_token_type_ids, 2)

      saida_modelo = model(inputs_embeds=input_ids, attention_mask=input_masks, token_type_ids=input_token_type_ids.long(), labels=labels.long())
      #saida_modelo = model(inputs_embeds=input_ids, attention_mask=input_masks, labels=labels.long())

      loss, logits = saida_modelo[0], saida_modelo[1]

      optim.zero_grad()
      loss.backward(retain_graph=True)
#      loss.backward()
      optim.step()

      train_running_loss += loss.item()

      softmax_predictions = torch.nn.functional.softmax(logits, dim=1)
      sigmoid_predictions = torch.nn.functional.sigmoid(logits)
      train_hits_softmax += acc_calc(softmax_predictions, labels)
      train_hits_sigmoid += acc_calc(sigmoid_predictions, labels)

      train_precision_softmax += metric1.compute(predictions=softmax_predictions.argmax(axis=1), references=labels)["precision"]
      train_recall_softmax += metric2.compute(predictions=softmax_predictions.argmax(axis=1), references=labels)["recall"]
      train_f1_softmax += metric3.compute(predictions=softmax_predictions.argmax(axis=1), references=labels)["f1"]
      train_f1_macro_softmax += metric3.compute(predictions=softmax_predictions.argmax(axis=1), references=labels,average ="macro")["f1"]

      train_precision_sigmoid += metric1.compute(predictions=sigmoid_predictions.argmax(axis=1), references=labels)["precision"]
      train_recall_sigmoid += metric2.compute(predictions=sigmoid_predictions.argmax(axis=1), references=labels)["recall"]
      train_f1_sigmoid += metric3.compute(predictions=sigmoid_predictions.argmax(axis=1), references=labels)["f1"]
      train_f1_macro_sigmoid += metric3.compute(predictions=sigmoid_predictions.argmax(axis=1), references=labels,average ="macro")["f1"]

      train_bar.update(1)

      num_train_examples += features.shape[0]
      train_iteracoes = train_iteracoes + 1

      scheduler.step()

      if (batch_number + 1) % steps_per_epoch == 0:
        train_bar.close()
        break


    model.save_pretrained(f'experiment_1_and_2_epoch_{i}')

    train_acc_softmax = torch.true_divide(train_hits_softmax, num_train_examples)
    train_acc_sigmoid = torch.true_divide(train_hits_sigmoid, num_train_examples)

    train_precision_softmax_final = torch.true_divide(train_precision_softmax, train_iteracoes)
    train_recall_softmax_final = torch.true_divide(train_recall_softmax, train_iteracoes)
    train_f1_softmax_final = torch.true_divide(train_f1_softmax, train_iteracoes)
    train_f1_macro_softmax_final = torch.true_divide(train_f1_macro_softmax, train_iteracoes)

    train_precision_sigmoid_final = torch.true_divide(train_precision_sigmoid, train_iteracoes)
    train_recall_sigmoid_final = torch.true_divide(train_recall_sigmoid, train_iteracoes)
    train_f1_sigmoid_final = torch.true_divide(train_f1_sigmoid, train_iteracoes)
    train_f1_macro_sigmoid_final = torch.true_divide(train_f1_macro_sigmoid, train_iteracoes)

    print(f"EPOCH SUMMARY - {i + 1}  TRAIN: \t Train loss: {train_running_loss} ")
    print(f"EPOCH SUMMARY - {i + 1}  TRAIN SOFTMAX: \t Train Acc softmax: {train_acc_softmax} \t Train precision softmax: {train_precision_softmax_final} \t Train recall softmax: {train_recall_softmax_final}  \t Train F1 softmax: {train_f1_softmax_final} \t Train F1 macro softmax: {train_f1_macro_softmax_final} ")
    print(f"EPOCH SUMMARY - {i + 1}  TRAIN SIGMOID: \t Train Acc sigmoid: {train_acc_sigmoid} \t Train precision sigmoid: {train_precision_sigmoid_final} \t Train recall sigmoid: {train_recall_sigmoid_final}  \t Train F1 sigmoid: {train_f1_sigmoid_final} \t Train F1 macro sigmoid: {train_f1_macro_sigmoid_final} ")



#    train_acc_softmax = torch.true_divide(train_hits_softmax, num_train_examples)
#    train_acc_sigmoid = torch.true_divide(train_hits_sigmoid, num_train_examples)
#    print(f"EPOCH SUMMARY - {i + 1} \t Train loss: {train_running_loss} \t Train Acc Softmax: {train_acc_softmax} ")


  return train_acc_softmax

In [ ]:
def TestModel(val_loader, test_loader):

  #Setup do treino
  #Como sugerido pelo artigo original do Bert, o modelo foi treinado usando 3 épocas.
  #Além disso, foram configurados 200 passos por época
  epochs = 1

  epoch_validation_samples = len(val_loader)
  epoch_test_samples = len(test_loader)

  print("Tamanho Validação: " + str(epoch_validation_samples))
  print("Tamanho Test: " + str(epoch_test_samples))
  print("Grad: " + str(GRAD_PARAMETER))


  model = BertForSequenceClassification.from_pretrained(model_name).to(device)
  for param in model.base_model.parameters():
    param.requires_grad = GRAD_PARAMETER

  loss_func = torch.nn.CrossEntropyLoss()
  optim = torch.optim.Adam(model.parameters())

  acc_calc = lambda output, labels: (labels == output.argmax(axis=1)).sum()
  scheduler = torch.optim.lr_scheduler.ExponentialLR(optim, 0.9997)

  epoch_metadata = []

  for i in range(epochs):
    val_iteracoes = 0
    test_iteracoes = 0
    num_val_examples = 0
    num_test_examples = 0

    val_hits_softmax = 0
    val_hits_sigmoid = 0
    test_hits_softmax = 0
    test_hits_sigmoid = 0

    val_precision_softmax = 0
    val_recall_softmax = 0
    val_f1_softmax = 0
    val_f1_macro_softmax = 0
    val_precision_sigmoid = 0
    val_recall_sigmoid = 0
    val_f1_sigmoid = 0
    val_f1_macro_sigmoid = 0

    test_precision_softmax = 0
    test_recall_softmax = 0
    test_f1_softmax = 0
    test_f1_macro_softmax = 0
    test_precision_sigmoid = 0
    test_recall_sigmoid = 0
    test_f1_sigmoid = 0
    test_f1_macro_sigmoid = 0


    #Barras para monitorar o teste
    val_bar = tqdm(total=epoch_validation_samples, desc=f'Val', unit='samples', position=0, leave=True)
    test_bar = tqdm(total=epoch_test_samples, desc=f'Test', unit='samples', position=0, leave=True)

    model.eval()

    for batch_number, (features, labels) in enumerate(val_loader):
      with torch.no_grad():

        val_running_loss = 0

        input_ids, input_masks, input_token_type_ids = features[:, 0, :], features[:, 1, :], features[:, 2, :]
        input_masks = input_masks[..., :1]
        input_masks = torch.squeeze(input_masks, 2)
        input_token_type_ids = input_token_type_ids[..., :1]
        input_token_type_ids = torch.squeeze(input_token_type_ids, 2)

        #saida_modelo = model(input_ids, input_masks, labels=labels.long())
        saida_modelo = model(inputs_embeds=input_ids, attention_mask=input_masks, token_type_ids=input_token_type_ids.long(), labels=labels.long())

        loss, logits = saida_modelo[0], saida_modelo[1]
        val_running_loss += loss.item()

        softmax_predictions = torch.nn.functional.softmax(logits, dim=1)
        sigmoid_predictions = torch.nn.functional.sigmoid(logits)
        val_hits_softmax += acc_calc(softmax_predictions, labels)
        val_hits_sigmoid += acc_calc(sigmoid_predictions, labels)

        val_precision_softmax += metric1.compute(predictions=softmax_predictions.argmax(axis=1), references=labels)["precision"]
        val_recall_softmax += metric2.compute(predictions=softmax_predictions.argmax(axis=1), references=labels)["recall"]
        val_f1_softmax += metric3.compute(predictions=softmax_predictions.argmax(axis=1), references=labels)["f1"]
        val_f1_macro_softmax += metric3.compute(predictions=softmax_predictions.argmax(axis=1), references=labels,average ="macro")["f1"]

        val_precision_sigmoid += metric1.compute(predictions=sigmoid_predictions.argmax(axis=1), references=labels)["precision"]
        val_recall_sigmoid += metric2.compute(predictions=sigmoid_predictions.argmax(axis=1), references=labels)["recall"]
        val_f1_sigmoid += metric3.compute(predictions=sigmoid_predictions.argmax(axis=1), references=labels)["f1"]
        val_f1_macro_sigmoid += metric3.compute(predictions=sigmoid_predictions.argmax(axis=1), references=labels,average ="macro")["f1"]


        num_val_examples += features.shape[0]
        val_iteracoes = val_iteracoes + 1

        val_bar.update(1)

        if (batch_number + 1) % epoch_validation_samples == 0:
          val_bar.close()
          break


    for batch_number, (features, labels) in enumerate(test_loader):
      with torch.no_grad():

        test_running_loss = 0

        input_ids, input_masks, input_token_type_ids = features[:, 0, :], features[:, 1, :], features[:, 2, :]
        input_masks = input_masks[..., :1]
        input_masks = torch.squeeze(input_masks, 2)
        input_token_type_ids = input_token_type_ids[..., :1]
        input_token_type_ids = torch.squeeze(input_token_type_ids, 2)

        #saida_modelo = model(input_ids, input_masks, labels=labels.long())
        saida_modelo = model(inputs_embeds=input_ids, attention_mask=input_masks, token_type_ids=input_token_type_ids.long(), labels=labels.long())

        loss, logits = saida_modelo[0], saida_modelo[1]
        test_running_loss += loss.item()


        softmax_predictions = torch.nn.functional.softmax(logits, dim=1)
        sigmoid_predictions = torch.nn.functional.sigmoid(logits)
        test_hits_softmax += acc_calc(softmax_predictions, labels)
        test_hits_sigmoid += acc_calc(sigmoid_predictions, labels)

        test_precision_softmax += metric1.compute(predictions=softmax_predictions.argmax(axis=1), references=labels)["precision"]
        test_recall_softmax += metric2.compute(predictions=softmax_predictions.argmax(axis=1), references=labels)["recall"]
        test_f1_softmax += metric3.compute(predictions=softmax_predictions.argmax(axis=1), references=labels)["f1"]
        test_f1_macro_softmax += metric3.compute(predictions=softmax_predictions.argmax(axis=1), references=labels,average ="macro")["f1"]

        test_precision_sigmoid += metric1.compute(predictions=sigmoid_predictions.argmax(axis=1), references=labels)["precision"]
        test_recall_sigmoid += metric2.compute(predictions=sigmoid_predictions.argmax(axis=1), references=labels)["recall"]
        test_f1_sigmoid += metric3.compute(predictions=sigmoid_predictions.argmax(axis=1), references=labels)["f1"]
        test_f1_macro_sigmoid += metric3.compute(predictions=sigmoid_predictions.argmax(axis=1), references=labels,average ="macro")["f1"]

        num_test_examples += features.shape[0]
        test_iteracoes = test_iteracoes + 1

        test_bar.update(1)

        if (batch_number + 1) % epoch_test_samples == 0:
          test_bar.close()
          break

    val_acc_softmax = torch.true_divide(val_hits_softmax, num_val_examples)
    val_acc_sigmoid = torch.true_divide(val_hits_sigmoid, num_val_examples)
    test_acc_softmax = torch.true_divide(test_hits_softmax, num_test_examples)
    test_acc_sigmoid = torch.true_divide(test_hits_sigmoid, num_test_examples)


    val_precision_softmax_final = torch.true_divide(val_precision_softmax, val_iteracoes)
    val_recall_softmax_final = torch.true_divide(val_recall_softmax, val_iteracoes)
    val_f1_softmax_final = torch.true_divide(val_f1_softmax, val_iteracoes)
    val_f1_macro_softmax_final = torch.true_divide(val_f1_macro_softmax, val_iteracoes)

    val_precision_sigmoid_final = torch.true_divide(val_precision_sigmoid, val_iteracoes)
    val_recall_sigmoid_final = torch.true_divide(val_recall_sigmoid, val_iteracoes)
    val_f1_sigmoid_final = torch.true_divide(val_f1_sigmoid, val_iteracoes)
    val_f1_macro_sigmoid_final = torch.true_divide(val_f1_macro_sigmoid, val_iteracoes)

    test_precision_softmax_final = torch.true_divide(test_precision_softmax, test_iteracoes)
    test_recall_softmax_final = torch.true_divide(test_recall_softmax, test_iteracoes)
    test_f1_softmax_final = torch.true_divide(test_f1_softmax, test_iteracoes)
    test_f1_macro_softmax_final = torch.true_divide(test_f1_macro_softmax, test_iteracoes)

    test_precision_sigmoid_final = torch.true_divide(test_precision_sigmoid, test_iteracoes)
    test_recall_sigmoid_final = torch.true_divide(test_recall_sigmoid, test_iteracoes)
    test_f1_sigmoid_final = torch.true_divide(test_f1_sigmoid, test_iteracoes)
    test_f1_macro_sigmoid_final = torch.true_divide(test_f1_macro_sigmoid, test_iteracoes)


#    print('Val precision softmax será')
#    print(val_precision_softmax)
#    print('Número de testes examples')
#    print(val_iteracoes)

    print(f"EPOCH SUMMARY - {i + 1}  VALIDAÇÃO: \t Val loss: {val_running_loss} ")
    print(f"EPOCH SUMMARY - {i + 1}  VALIDAÇÃO SOFTMAX: \t Val Acc softmax: {val_acc_softmax} \t Val precision softmax: {val_precision_softmax_final} \t Val recall softmax: {val_recall_softmax_final}  \t Val F1 softmax: {val_f1_softmax_final} \t Val F1 macro softmax: {val_f1_macro_softmax_final} ")
    print(f"EPOCH SUMMARY - {i + 1}  VALIDAÇÃO SIGMOID: \t Val Acc sigmoid: {val_acc_sigmoid} \t Val precision sigmoid: {val_precision_sigmoid_final} \t Val recall sigmoid: {val_recall_sigmoid_final}  \t Val F1 sigmoid: {val_f1_sigmoid_final} \t Val F1 macro sigmoid: {val_f1_macro_sigmoid_final} ")

    print(f"EPOCH SUMMARY - {i + 1}  TEST: \t test loss: {test_running_loss} ")
    print(f"EPOCH SUMMARY - {i + 1}  TEST SOFTMAX: \t Test Acc softmax: {test_acc_softmax} \t Test precision softmax: {test_precision_softmax_final} \t Test recall softmax: {test_recall_softmax_final}  \t Test F1 softmax: {test_f1_softmax_final} \t Test F1 macro softmax: {test_f1_macro_softmax_final} ")
    print(f"EPOCH SUMMARY - {i + 1}  TEST SIGMOID: \t Test Acc sigmoid: {test_acc_sigmoid} \t Test precision sigmoid: {test_precision_sigmoid_final} \t Test recall sigmoid: {test_recall_sigmoid_final}  \t Test F1 sigmoid: {test_f1_sigmoid_final} \t Test F1 macro sigmoid: {test_f1_macro_sigmoid_final} ")


#    print(f"EPOCH SUMMARY - {i + 1}  \t Val loss: {val_running_loss} \t Val Acc Softmax: {val_acc_softmax} \t Val Acc Sigmoid: {val_acc_sigmoid}  \t Test loss: {test_running_loss} \t Test Acc Softmax: {test_acc_softmax} \t Test Acc Sigmoid: {test_acc_sigmoid}")
#   print(f"EPOCH SUMMARY - {i + 1}  \t Val precision softmax: {val_precision_softmax_final} \t Val precision sigmoid: {val_precision_sigmoid_final} \t Val recall softmax: {val_recall_softmax_final}  \t Val recall sigmoid: {val_recall_sigmoid_final} \t Val F1 softmax: {val_f1_softmax_final} \t Val F1 sigmoid: {val_f1_sigmoid_final} \t Val F1 macro softmax: {val_f1_macro_softmax_final} \t Val F1 macro sigmoid: {val_f1_macro_sigmoid_final}")
#    print(f"EPOCH SUMMARY - {i + 1}  \t Test precision softmax: {test_precision_softmax_final} \t Test precision sigmoid: {test_precision_sigmoid_final} \t Test recall softmax: {test_recall_softmax_final}  \t Test recall sigmoid: {test_recall_sigmoid_final} \t Test F1 softmax: {test_f1_softmax_final} \t Test F1 sigmoid: {test_f1_sigmoid_final} \t Test F1 macro softmax: {test_f1_macro_softmax_final} \t Test F1 macro sigmoid: {test_f1_macro_sigmoid_final}")


  return test_acc_softmax

In [ ]:
bert = BertModel.from_pretrained('neuralmind/bert-base-portuguese-cased')
#Executar se o modelo for testado
if (Execucao != 'TrainModel'):
  val_df = pd.read_xml(conjunto_teste)
  testing_df = pd.read_xml(conjunto_teste2)
  #Pré processamento dos dados para montar a lista de tokens
  for i in val_df.index:
    tokens = list(val_df['t_tokenized'][i].split(" "))
    val_df['t_tokenized'][i] = tokens

    tokens = list(val_df['h_tokenized'][i].split(" "))
    val_df['h_tokenized'][i] = tokens


  #Pré processamento dos dados para montar a lista de tokens
  for i in testing_df.index:
    tokens = list(testing_df['t_tokenized'][i].split(" "))
    testing_df['t_tokenized'][i] = tokens

    tokens = list(testing_df['h_tokenized'][i].split(" "))
    testing_df['h_tokenized'][i] = tokens

  ##### Iniciando a montagem o conjunto de val
  sentences_pair = val_df[['t_tokenized', 'h_tokenized']]
  setences_pair_val = [tuple(x) for x in sentences_pair.to_numpy()]

  #Tokenização do dataset usando o modelo neuralmind/bert-base-portuguese-cased da Neuralmind, retornando tensores do Pytorch, com padding = True, com truncation = True e tamanho máximo da sentença conforme parâmetro.
  df_tokenized = tokenizer.batch_encode_plus(setences_pair_val, return_tensors='pt', padding=True, truncation=True, max_length=MAX_LENGTH, is_split_into_words=True)
  print(df_tokenized['input_ids'].shape, df_tokenized['attention_mask'].shape)

  #X = torch.stack((df_tokenized['input_ids'], df_tokenized['attention_mask'], df_tokenized['token_type_ids']), dim=0)
  #y = torch.Tensor(val_df['redundancy'].to_numpy())

  val_embeddings = bert.embeddings.word_embeddings(df_tokenized['input_ids'])
  attention_mask_representation = df_tokenized['attention_mask']
  token_type_ids_representation = df_tokenized['token_type_ids']
  attention_mask_representation = attention_mask_representation[:, :, None]
  attention_mask_representation = attention_mask_representation.expand(-1, -1, 768)
  token_type_ids_representation = token_type_ids_representation[:, :, None]
  token_type_ids_representation = token_type_ids_representation.expand(-1, -1, 768)

  X = torch.stack((val_embeddings, attention_mask_representation, token_type_ids_representation), dim=0)
  y = torch.Tensor(val_df['redundancy'].to_numpy())

  dataset = TextDataset(X, y)
  val_split = dataset
  #9. O DataLoader do Pytorch fica responsável por alimentar o medulo com os dados do treinamento.
  #A cada vez que o Dataloader pegar um batch (batch sizer = 16), ele não sorteia as instância (shuffle = False)
  val_loader = torch.utils.data.DataLoader(val_split, batch_size=BATCH_SIZE, shuffle=True)

  ##### Finalizando a montagem o conjunto de val


  ##### Iniciando a montagem o conjunto de teste
  sentences_pair = testing_df[['t_tokenized', 'h_tokenized']]
  setences_pair_testing = [tuple(x) for x in sentences_pair.to_numpy()]

  #Tokenização do dataset usando o modelo neuralmind/bert-base-portuguese-cased da Neuralmind, retornando tensores do Pytorch, com padding = True, com truncation = True e tamanho máximo da sentença conforme parâmetro.
  df_tokenized = tokenizer.batch_encode_plus(setences_pair_testing, return_tensors='pt', padding=True, truncation=True, max_length=MAX_LENGTH, is_split_into_words=True)
  print(df_tokenized['input_ids'].shape, df_tokenized['attention_mask'].shape)

  #X = torch.stack((df_tokenized['input_ids'], df_tokenized['attention_mask'], df_tokenized['token_type_ids']), dim=0)
  #y = torch.Tensor(testing_df['redundancy'].to_numpy())

  test_embeddings = bert.embeddings.word_embeddings(df_tokenized['input_ids'])
  attention_mask_representation = df_tokenized['attention_mask']
  token_type_ids_representation = df_tokenized['token_type_ids']
  attention_mask_representation = attention_mask_representation[:, :, None]
  attention_mask_representation = attention_mask_representation.expand(-1, -1, 768)
  token_type_ids_representation = token_type_ids_representation[:, :, None]
  token_type_ids_representation = token_type_ids_representation.expand(-1, -1, 768)

  X = torch.stack((test_embeddings, attention_mask_representation, token_type_ids_representation), dim=0)
  y = torch.Tensor(testing_df['redundancy'].to_numpy())

  dataset = TextDataset(X, y)
  test_split = dataset
  #9. O DataLoader do Pytorch fica responsável por alimentar o medulo com os dados do treinamento.
  #A cada vez que o Dataloader pegar um batch (batch sizer = 16), ele não sorteia as instância (shuffle = False)
  test_loader = torch.utils.data.DataLoader(test_split, batch_size=BATCH_SIZE, shuffle=True)

  ##### Finalizando a montagem o conjunto de teste


#Executar se o modelo for treinado
if (Execucao != 'TestModel'):
  training_df = pd.read_xml(conjunto_treino)
  #Pré processamento dos dados para montar a lista de tokens
  for i in training_df.index:
    tokens = list(training_df['t_tokenized'][i].split(" "))
    training_df['t_tokenized'][i] = tokens

    tokens = list(training_df['h_tokenized'][i].split(" "))
    training_df['h_tokenized'][i] = tokens

  ##### Iniciando a montagem do conjunto de treino
  sentences_pair = training_df[['t_tokenized', 'h_tokenized']]
  setences_pair_training = [tuple(x) for x in sentences_pair.to_numpy()]

  #Tokenização do dataset usando o modelo neuralmind/bert-base-portuguese-cased da Neuralmind, retornando tensores do Pytorch, com padding = True, com truncation = True e tamanho )
  #máximo da sentença conforme parâmetro.
  df_tokenized = tokenizer.batch_encode_plus(setences_pair_training, return_tensors='pt', padding=True, truncation=True, max_length=MAX_LENGTH, is_split_into_words=True)
  #print(df_tokenized['input_ids'].shape, df_tokenized['attention_mask'].shape)
  attention_mask_representation = df_tokenized['attention_mask']
  token_type_id_representation = df_tokenized['token_type_ids']




Streaming output truncated to the last 5000 lines.
<ipython-input-63-9edbdee84eeb>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testing_df['t_tokenized'][i] = tokens
<ipython-input-63-9edbdee84eeb>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testing_df['h_tokenized'][i] = tokens
<ipython-input-63-9edbdee84eeb>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testing_df['t_tokenized'][i] = tokens
<ipython-input-63-9edbdee84e

torch.Size([2000, 82]) torch.Size([2000, 82])
torch.Size([1890, 68]) torch.Size([1890, 68])


In [ ]:
if (POS_PARAMETER == True):
  if (Execucao != 'TestModel'):
    print("Execução com PoS")
    #### Executar esse bloco para experimento com PoS ####
    index = 0
    errors_count = []

    #O t são os tokens de cada sentença. Cada ocorrência desse for é uma sentença diferente

    for t in df_tokenized['input_ids']:

        sentence_pos_aligned = ''
        first_sentence = 0
        error_sentence = 0
        sentence_evaluated = training_df['t'][index]

        for sentence_word_id in df_tokenized.word_ids(index):
          if sentence_word_id is not None:
              sentence_pos_aligned = sentence_pos_aligned + complete_sentence_pos[sentence_word_id] + ' '
          else:
            if first_sentence == 0:
              complete_sentence_pos = training_df['t1'][index].split()
              first_sentence = 1
              error_sentence = 0
              sentence_evaluated = training_df['h'][index]
            else:
              if first_sentence == 1:
                training_df.loc[index, 't1'] = sentence_pos_aligned
                sentence_pos_aligned = ''
                complete_sentence_pos = training_df['h1'][index].split()
                first_sentence = 2

        training_df.loc[index, 'h1'] = sentence_pos_aligned
        index = index + 1

    ##### Finalizando o alinhamento das etiquetas PoS com as palavras


    ##### Iniciando a geração dos embeddings e soma dos embeddings dos tokens com embeddings das etiquetas PoS
    #bert = BertModel.from_pretrained('neuralmind/bert-base-portuguese-cased')
    token_embeddings = bert.embeddings.word_embeddings(df_tokenized['input_ids'])


    for i in training_df.index:
      tokens = list(training_df['t1'][i].split(" "))
      training_df['t1'][i] = tokens

      tokens = list(training_df['h1'][i].split(" "))
      training_df['h1'][i] = tokens

    pos_sentences_pair = training_df[['t1', 'h1']]
    pos_setences_pair_training = [tuple(x) for x in pos_sentences_pair.to_numpy()]


    #Adicionando as tags PoS ao tokenizador para que ele não as quebre em sub palavras. A lista completa de tags foi obtida no manual PoS tags: https://drive.google.com/file/d/1BddPswn-_Ioo-A5GsldA1cO1kqbcCahb/view
    tokenizer.add_tokens(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X'])
    bert.resize_token_embeddings(len(tokenizer))

    df_tokenized = tokenizer.batch_encode_plus(pos_setences_pair_training, return_tensors='pt', padding=True, truncation=True, max_length=MAX_LENGTH, is_split_into_words=True)
    pos_embeddings = bert.embeddings.word_embeddings(df_tokenized['input_ids'])

    input_representation = token_embeddings + pos_embeddings

    ##### Finalizando a geração dos embeddings e soma dos embeddings dos tokens com embeddings das etiquetas PoS



In [ ]:
if (POS_PARAMETER == False):
  if (Execucao != 'TestModel'):

    print("Execução sem PoS")
    #### Executar esse bloco para experimento sem PoS ####

    ##### Iniciando a geração dos embeddings
    bert = BertModel.from_pretrained('neuralmind/bert-base-portuguese-cased')
    token_embeddings = bert.embeddings.word_embeddings(df_tokenized['input_ids'])

    input_representation = token_embeddings

  ##### Finalizando a geração dos embeddings


In [ ]:
if (Execucao != 'TestModel'):
  attention_mask_representation = attention_mask_representation[:, :, None]
  attention_mask_representation = attention_mask_representation.expand(-1, -1, 768)

  token_type_id_representation = token_type_id_representation[:, :, None]
  token_type_id_representation = token_type_id_representation.expand(-1, -1, 768)

In [ ]:
# Delete the old DataFrame
import gc
#del df_tokenized
#del tokens, sentences_pair, dataset
del bert

if (Execucao != 'TrainModel'):
    del test_split, setences_pair_testing

if (Execucao != 'TestModel'):
    del setences_pair_training, token_embeddings

if (POS_PARAMETER == True):
  if (Execucao != 'TestModel'):
    del index, errors_count, sentence_pos_aligned,first_sentence, error_sentence, sentence_evaluated, sentence_word_id, complete_sentence_pos
    del pos_embeddings

# Perform garbage collection
gc.collect()

0

In [ ]:
print("#############################################")
print("CONFIGURAÇÕES DA EXECUÇÃO")
print("Tipo de execução = " + Execucao)
print("Conjunto de treino = " + conjunto_treino)
print("Conjunto de teste 2 = " + conjunto_teste2)
print("Conjunto de teste = " + conjunto_teste)
print("Modelo a ser usado nas vaidações e teste = " + model_name)
print("POS_PARAMETER = " + str(POS_PARAMETER))
print("GRAD_PARAMETER = " + str(GRAD_PARAMETER))
print("Épocas = " + str(epochs))
print("MAX_LENGTH = " + str(MAX_LENGTH))
print("BATCH_SIZE = " + str(BATCH_SIZE))
print("#############################################")


if (Execucao != 'TestModel'):
  X = torch.stack((input_representation, attention_mask_representation, token_type_id_representation), dim=0)
  y = torch.Tensor(training_df['redundancy'].to_numpy())

  dataset = TextDataset(X, y)

  train_split = dataset
  train_loader = torch.utils.data.DataLoader(train_split, batch_size=BATCH_SIZE, shuffle=True)

  del X, y, dataset, train_split, input_representation, attention_mask_representation, token_type_id_representation, training_df
  gc.collect()


if (Execucao == 'TrainTestModel'):
  accuracy = TrainTestModel(train_loader, val_loader, test_loader)

if (Execucao == 'TrainModel'):
  accuracy = TrainModel(train_loader)

if (Execucao == 'TestModel'):
 # del val_df, testing_df
  accuracy = TestModel(val_loader, test_loader)



print("ACURÁCIA DO EXPERIMENTO = ")
print(accuracy)




#############################################
CONFIGURAÇÕES DA EXECUÇÃO
Tipo de execução = TestModel
Conjunto de treino = assin-trv1-pos.xml
Conjunto de validação = assin2-t2-pos.xml
Conjunto de teste = assin-t1-pos.xml
Modelo a ser usado nas vaidações e teste = experiment_1_and_2_epoch_2
POS_PARAMETER = True
GRAD_PARAMETER = True
Épocas = 3
MAX_LENGTH = 512
BATCH_SIZE = 16
#############################################
Tamanho Validação: 125
Tamanho Test: 119
Grad: True


Test:   0%|          | 0/119 [00:00<?, ?samples/s]/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13

EPOCH SUMMARY - 1  VALIDAÇÃO: 	 Val loss: 0.45597946643829346 
EPOCH SUMMARY - 1  VALIDAÇÃO SOFTMAX: 	 Val Acc softmax: 0.7590000629425049 	 Val precision softmax: 0.0 	 Val recall softmax: 0.0  	 Val F1 softmax: 0.0 	 Val F1 macro softmax: 0.44909361004829407 
EPOCH SUMMARY - 1  VALIDAÇÃO SIGMOID: 	 Val Acc sigmoid: 0.7590000629425049 	 Val precision sigmoid: 0.0 	 Val recall sigmoid: 0.0  	 Val F1 sigmoid: 0.0 	 Val F1 macro sigmoid: 0.44909361004829407 
EPOCH SUMMARY - 1  TEST: 	 test loss: 0.34523603320121765 
EPOCH SUMMARY - 1  TEST SOFTMAX: 	 Test Acc softmax: 0.4513227641582489 	 Test precision softmax: 0.0 	 Test recall softmax: 0.0  	 Test F1 softmax: 0.0 	 Test F1 macro softmax: 0.31180059909820557 
EPOCH SUMMARY - 1  TEST SIGMOID: 	 Test Acc sigmoid: 0.4513227641582489 	 Test precision sigmoid: 0.0 	 Test recall sigmoid: 0.0  	 Test F1 sigmoid: 0.0 	 Test F1 macro sigmoid: 0.31180059909820557 
ACURÁCIA DO EXPERIMENTO = 
tensor(0.4513, device='cuda:0')
